In [4]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai_api_key = os.environ['OPENAI_API_KEY']

from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)


def gen_gpt_messages(prompt):
    messages = [{"role": "user", "content": prompt}]
    return messages


def get_completion(prompt, model="gpt-3.5-turbo", temperature = 0):
    response = client.chat.completions.create(
        model=model,
        messages=gen_gpt_messages(prompt),
        temperature=temperature,
    )
    if len(response.choices) > 0:
        return response.choices[0].message.content
    return "generate answer error"

### 文本翻译

In [5]:
# 翻译
prompt = f"""
将以下中文翻译成西班牙语：\
```您好，我想订购一个搅拌机。```
"""
reponse = get_completion(prompt)
print(reponse)

Hola, me gustaría ordenar una batidora.


In [6]:
# 识别语种
prompt = f"""
请告诉我以下文本是什么语种:
```Combien coûte le lampadaire?```
"""
response = get_completion(prompt)
print(response)

这是法语。


In [7]:
#多语种翻译
prompt = f"""
请将以下文本分别翻译成中文、英文、法语和西班牙语: 
```I want to order a basketball.```
"""
response = get_completion(prompt)
print(response)

中文：我想订购一个篮球。
英文：I want to order a basketball.
法语：Je veux commander un ballon de basket.
西班牙语：Quiero pedir un balón de baloncesto.


In [8]:
#语气转换
prompt = f"""
请将以下文本翻译成中文，分别展示成正式与非正式两种语气:
```Would you like to order a pillow?```
"""
response = get_completion(prompt)
print(response)

正式：您想要订购一个枕头吗？
非正式：你想要订购一个枕头吗？


In [9]:
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                             # My screen is flashing
]

#通用翻译器（识别语言、翻译成对应语言）
import time
for issue in user_messages:
    time.sleep(20)
    prompt = f"告诉我一下文本是什么语种，直接输出语种，如法语，无需输出标点符号```{issue}```"
    lang = get_completion(prompt)
    print(f"原始信息({lang}):{issue}\n")

    prompt = f"""
    将以下消息分别翻译成英文和中文，并写成
    中文翻译：xxx
    英文翻译：yyy
    的格式:
    ```{issue}```
    """
    response = get_completion(prompt)
    print(response,"\n=======================")

原始信息(法语):La performance du système est plus lente que d'habitude.

中文翻译：系统性能比平时慢。
英文翻译：The system performance is slower than usual. 
原始信息(西班牙语):Mi monitor tiene píxeles que no se iluminan.

中文翻译：我的显示器有一些像素不亮。
英文翻译：My monitor has pixels that do not light up. 
原始信息(意大利语):Il mio mouse non funziona

中文翻译：我的鼠标不工作
英文翻译：My mouse is not working 
原始信息(波兰语。):Mój klawisz Ctrl jest zepsuty

中文翻译：我的Ctrl键坏了
英文翻译：My Ctrl key is broken 
原始信息(中文):我的屏幕在闪烁

中文翻译：我的屏幕在闪烁
英文翻译：My screen is flickering 


### 语气与写作风格调整

In [10]:
prompt = f"""
将以下文本转换成商务信函的格式，更正式一些: 
```小老弟，我小羊，上回你说咱部门要采购的显示器是多少寸来着？```
"""
response = get_completion(prompt)
print(response)

尊敬的先生/女士，

我是小羊，特此致函。上次您提到我们部门需要采购显示器，您能告诉我需要多大尺寸的显示器吗？

谢谢您的合作。

此致
敬礼

[您的姓名]


### 文本格式转换

In [11]:
data_json = { "resturant employees" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

In [12]:
prompt = f"""
将以下Python字典从Json转换到HTML表格，保留表格标题和列名：{data_json}
"""
response = get_completion(prompt)
print(response)

<!DOCTYPE html>
<html>
<head>
    <title>Restaurant Employees</title>
</head>
<body>

    <table border="1">
        <tr>
            <th>Name</th>
            <th>Email</th>
        </tr>
        <tr>
            <td>Shyam</td>
            <td>shyamjaiswal@gmail.com</td>
        </tr>
        <tr>
            <td>Bob</td>
            <td>bob32@gmail.com</td>
        </tr>
        <tr>
            <td>Jai</td>
            <td>jai87@gmail.com</td>
        </tr>
    </table>

</body>
</html>


### 拼写及语法纠正

In [13]:
text = [ 
  "The girl with the black and white puppies have a ball.",  # The girl has a ball.
  "Yolanda has her notebook.", # ok
  "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
  "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
  "Your going to need you’re notebook.",  # Homonyms
  "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
  "This phrase is to cherck chatGPT for spelling abilitty"  # spelling
]

In [14]:
for i in range(len(text)):
    time.sleep(20)
    prompt = f"""请校对并更正以下文本，注意纠正文本保持原始语种，无需输出原始文本。
    如果您没有发现任何错误，请说“未发现错误”。
    
    例如：
    输入：I are happy.
    输出：I am happy.
    ```{text[i]}```"""
    response = get_completion(prompt)
    print(i, response)

0 The girl with the black and white puppies has a ball.
1 未发现错误。
2 未发现错误。
3 Their goes my freedom. There going to bring their suitcases.
4 Your going to need your notebook.
5 未发现错误。
6 未发现错误。


In [15]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""

In [16]:
prompt = f"校对并更正以下商品评论：```{text}```"
response = get_completion(prompt)
print(response)

I got this for my daughter for her birthday because she keeps taking mine from my room. Yes, adults also like pandas too. She takes it everywhere with her, and it's super soft and cute. One of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. It's a bit small for what I paid for it though. I think there might be other options that are bigger for the same price. It arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.


In [17]:
# !pip install redlines

from redlines import Redlines
from IPython.display import display, Markdown

diff = Redlines(text, response)
display(Markdown(diff.output_markdown))

<span style='color:red;font-weight:700;text-decoration:line-through;'>Got </span><span style='color:green;font-weight:700;'>I got </span>this for my daughter for her birthday <span style='color:red;font-weight:700;text-decoration:line-through;'>cuz </span><span style='color:green;font-weight:700;'>because </span>she keeps taking mine from my <span style='color:red;font-weight:700;text-decoration:line-through;'>room.  </span><span style='color:green;font-weight:700;'>room. </span>Yes, adults also like pandas <span style='color:red;font-weight:700;text-decoration:line-through;'>too.  </span><span style='color:green;font-weight:700;'>too. </span>She takes it everywhere with her, and it's super soft and <span style='color:red;font-weight:700;text-decoration:line-through;'>cute.  </span><span style='color:green;font-weight:700;'>cute. </span>One of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. It's a bit small for what I paid for it though. I think there might be other options that are bigger for the same <span style='color:red;font-weight:700;text-decoration:line-through;'>price.  </span><span style='color:green;font-weight:700;'>price. </span>It arrived a day earlier than expected, so I got to play with it myself before I gave it to my daughter.

### 综合样例
一段prompt同时进行文本翻译+拼写纠正+风格调整+格式转换

In [18]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""

In [19]:
prompt = f"""
针对以下三个反引号之间的英文评论文本，
首先进行拼写及语法纠错，
然后将其转化成中文，
再将其转化成优质淘宝评论的风格，从各种角度出发，分别说明产品的优点与缺点，并进行总结。
润色一下描述，使评论更具有吸引力。
输出结果格式为：
【优点】xxx \n
【缺点】xxx \n
【总结】xxx
注意，只需填写xxx部分，并分段输出。
将结果输出成Markdown格式。
```{text}```
"""
response = get_completion(prompt)
display(Markdown(response))

【优点】超级柔软和可爱，适合孩子和成年人喜欢熊猫的人

【缺点】耳朵有点不对称，有点小，性价比略低

【总结】虽然有一些小问题，但总体来说是一款受欢迎的熊猫玩具，适合送给喜欢熊猫的孩子或成年人。

#### Json摘要

In [20]:
messages = context.copy()
messages.append(
    {'role':'system', 'content':
    """创建上一个食品订单的json摘要。\
    逐项列出每件商品的价格，字段应该是1) 披萨，包括大小 2）配料表 3）饮料列表，包括大小 4）配菜列表包括大小
    5） 总价 你应该给我返回一个可解析的Json对象，包括上述字段
    """}
)
response = get_completion_from_messages(messages, temperature=0)
print(response)

NameError: name 'context' is not defined